# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/4. normalized-50/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (604, 26)
y_train: (604, 2)
x_test: (604, 26)
y_test: (604, 2)


# Tunable parameters

In [3]:
epochs = 3000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001
loss = keras.losses.mean_squared_error

# Autoencoder #1

In [4]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/34-autoencoder-1-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layer_size, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layer_size, activation='relu')(x)
    y = Dropout(dropout)(y)

    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Hidden layer units': layer_size,
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Epochs': epochs,
        'Batch Size': batch_size,
        'Loss': loss
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=loss, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [5]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search

In [6]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [10, 8]
dropouts = [0, 0.4]
param_grid = dict(
    layer_size=layers, 
    dropout=dropouts
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, x_test)))
grid_result = grid.fit(x_train, x_train)

Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV] dropout=0, layer_size=10 ........................................
Train on 604 samples, validate on 604 samples
Epoch 1/3000
604/604 [==============================] - 0s - loss: 0.1339 - acc: 0.0017 - val_loss: 0.1318 - val_acc: 0.0050
Epoch 2/3000
604/604 [==============================] - 0s - loss: 0.1322 - acc: 0.0017 - val_loss: 0.1302 - val_acc: 0.0050
Epoch 3/3000
604/604 [==============================] - 0s - loss: 0.1307 - acc: 0.0017 - val_loss: 0.1287 - val_acc: 0.0050
Epoch 4/3000
604/604 [==============================] - 0s - loss: 0.1292 - acc: 0.0017 - val_loss: 0.1273 - val_acc: 0.0050
Epoch 5/3000
604/604 [==============================] - 0s - loss: 0.1278 - acc: 0.0017 - val_loss: 0.1259 - val_acc: 0.0050
Epoch 6/3000
604/604 [==============================] - 0s - loss: 0.1264 - acc: 0.0017 - val_loss: 0.1246 - val_acc: 0.0050
Epoch 7/3000
604/604 [==============================] - 0s - loss: 0.1251 

604/604 [==============================] - 0s - loss: 0.0398 - acc: 0.6457 - val_loss: 0.0370 - val_acc: 0.6838
Epoch 65/3000
604/604 [==============================] - 0s - loss: 0.0392 - acc: 0.6457 - val_loss: 0.0364 - val_acc: 0.6838
Epoch 66/3000
604/604 [==============================] - 0s - loss: 0.0387 - acc: 0.6457 - val_loss: 0.0358 - val_acc: 0.6838
Epoch 67/3000
604/604 [==============================] - 0s - loss: 0.0381 - acc: 0.6457 - val_loss: 0.0353 - val_acc: 0.6838
Epoch 68/3000
604/604 [==============================] - 0s - loss: 0.0376 - acc: 0.6457 - val_loss: 0.0348 - val_acc: 0.6838
Epoch 69/3000
604/604 [==============================] - 0s - loss: 0.0371 - acc: 0.6457 - val_loss: 0.0343 - val_acc: 0.6838
Epoch 70/3000
604/604 [==============================] - 0s - loss: 0.0366 - acc: 0.6457 - val_loss: 0.0338 - val_acc: 0.6838
Epoch 71/3000
604/604 [==============================] - 0s - loss: 0.0361 - acc: 0.6457 - val_loss: 0.0333 - val_acc: 0.6838
Epoch 

604/604 [==============================] - 0s - loss: 0.0236 - acc: 0.6457 - val_loss: 0.0214 - val_acc: 0.6838
Epoch 129/3000
604/604 [==============================] - 0s - loss: 0.0235 - acc: 0.6457 - val_loss: 0.0213 - val_acc: 0.68380.631
Epoch 130/3000
604/604 [==============================] - 0s - loss: 0.0234 - acc: 0.6457 - val_loss: 0.0212 - val_acc: 0.6838
Epoch 131/3000
604/604 [==============================] - 0s - loss: 0.0232 - acc: 0.6457 - val_loss: 0.0211 - val_acc: 0.6838
Epoch 132/3000
604/604 [==============================] - 0s - loss: 0.0231 - acc: 0.6457 - val_loss: 0.0210 - val_acc: 0.6838
Epoch 133/3000
604/604 [==============================] - 0s - loss: 0.0230 - acc: 0.6457 - val_loss: 0.0209 - val_acc: 0.6838
Epoch 134/3000
604/604 [==============================] - 0s - loss: 0.0229 - acc: 0.6457 - val_loss: 0.0208 - val_acc: 0.6838
Epoch 135/3000
604/604 [==============================] - 0s - loss: 0.0228 - acc: 0.6457 - val_loss: 0.0207 - val_acc: 0

604/604 [==============================] - 0s - loss: 0.0173 - acc: 0.6457 - val_loss: 0.0159 - val_acc: 0.6838
Epoch 193/3000
604/604 [==============================] - 0s - loss: 0.0172 - acc: 0.6457 - val_loss: 0.0159 - val_acc: 0.6838
Epoch 194/3000
604/604 [==============================] - 0s - loss: 0.0171 - acc: 0.6457 - val_loss: 0.0158 - val_acc: 0.6838
Epoch 195/3000
604/604 [==============================] - 0s - loss: 0.0170 - acc: 0.6457 - val_loss: 0.0157 - val_acc: 0.6838
Epoch 196/3000
604/604 [==============================] - 0s - loss: 0.0170 - acc: 0.6457 - val_loss: 0.0157 - val_acc: 0.6838
Epoch 197/3000
604/604 [==============================] - 0s - loss: 0.0169 - acc: 0.6457 - val_loss: 0.0156 - val_acc: 0.6838
Epoch 198/3000
604/604 [==============================] - 0s - loss: 0.0168 - acc: 0.6457 - val_loss: 0.0155 - val_acc: 0.6838
Epoch 199/3000
604/604 [==============================] - 0s - loss: 0.0168 - acc: 0.6457 - val_loss: 0.0155 - val_acc: 0.6838

604/604 [==============================] - 0s - loss: 0.0136 - acc: 0.6540 - val_loss: 0.0127 - val_acc: 0.6954
Epoch 257/3000
604/604 [==============================] - 0s - loss: 0.0136 - acc: 0.6540 - val_loss: 0.0127 - val_acc: 0.6954
Epoch 258/3000
604/604 [==============================] - 0s - loss: 0.0135 - acc: 0.6540 - val_loss: 0.0127 - val_acc: 0.6954
Epoch 259/3000
604/604 [==============================] - 0s - loss: 0.0135 - acc: 0.6573 - val_loss: 0.0126 - val_acc: 0.6970
Epoch 260/3000
604/604 [==============================] - 0s - loss: 0.0135 - acc: 0.6589 - val_loss: 0.0126 - val_acc: 0.6987
Epoch 261/3000
604/604 [==============================] - 0s - loss: 0.0134 - acc: 0.6589 - val_loss: 0.0126 - val_acc: 0.6987
Epoch 262/3000
604/604 [==============================] - 0s - loss: 0.0134 - acc: 0.6589 - val_loss: 0.0125 - val_acc: 0.6987
Epoch 263/3000
604/604 [==============================] - 0s - loss: 0.0133 - acc: 0.6589 - val_loss: 0.0125 - val_acc: 0.6987

...

604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2689/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2690/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2691/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2692/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2693/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2694/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc: 0.8940
Epoch 2695/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0032 - val_acc:

604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2753/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2754/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2755/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2756/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2757/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2758/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2759/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8262 - val_loss: 0.0031 - val_acc:

604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2817/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2818/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2819/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2820/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2821/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2822/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2823/3000
604/604 [==============================] - 0s - loss: 0.0032 - acc: 0.8295 - val_loss: 0.0031 - val_acc:

604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2881/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2882/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2883/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2884/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2885/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2886/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2887/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc:

604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2945/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2946/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2947/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2948/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2949/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2950/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8278 - val_loss: 0.0031 - val_acc: 0.8940
Epoch 2951/3000
604/604 [==============================] - 0s - loss: 0.0031 - acc: 0.8295 - val_loss: 0.0031 - val_acc: